In [1]:
import urllib.request as req
from bs4 import BeautifulSoup
import sys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
import json
import pymysql

In [2]:
def crawling(input_data):
    res_next = ''
    i = 0
    # 'next' 의 Value가 None 일 경우 다음페이지에 후기 내용 X
    while res_next != None:
        url  = 'https://cache.29cm.co.kr/item-review/?item_no=650636&offset='+str(7*i)+'&limit=7'
        request = req.Request(url)
        response = req.urlopen(request)
        rescode = response.getcode()
        
        if(rescode==200):
            res_review = response.read()
            r_data =json.loads(res_review)
            res_next = r_data['next']
            print(res_next)
            
            for data in r_data['results']:
                input_data.append(data['contents']) 
            
        else:
            print('Error code:'+rescode)
            
        i += 1
        print(i)
    return input_data 

In [10]:
update_data = []
def add_data(update_data):
    data = []
    input_data = crawling(data)
    # 기존 데이터베이스에 저장된 데이터 불러오기
    conn = pymysql.connect(host='localhost', user ='root', password='root', db='collection', charset='utf8mb4')
    cur = conn.cursor()
    # 상품 번호 650636에 대한 데이터
    cur.execute('select review from 29cm where product_no = 650636')
    old_data = cur.fetchall()
    cur.close()
    conn.close()
    
    # 튜플인 부분 제거 후 리스트로 변환
    db_data = list(old_data)
    pre_db_data = [row[0] for row in db_data]
    
    #디비 데이터와 크롤링 데이터 비교 후 새로운 데이터 추출
    for new_data in input_data:
        if new_data not in pre_db_data:
            print(new_data)
            update_data.append(new_data)
            
    return update_data

In [11]:
update = add_data(update_data)

/item-review/?item_no=650636&limit=7&offset=7
1
/item-review/?item_no=650636&limit=7&offset=14
2
/item-review/?item_no=650636&limit=7&offset=21
3
/item-review/?item_no=650636&limit=7&offset=28
4
/item-review/?item_no=650636&limit=7&offset=35
5
/item-review/?item_no=650636&limit=7&offset=42
6
/item-review/?item_no=650636&limit=7&offset=49
7
/item-review/?item_no=650636&limit=7&offset=56
8
/item-review/?item_no=650636&limit=7&offset=63
9
/item-review/?item_no=650636&limit=7&offset=70
10
/item-review/?item_no=650636&limit=7&offset=77
11
/item-review/?item_no=650636&limit=7&offset=84
12
None
13
유광 무광 정말 많이 고민했는데 유광선택할 잘 한거 같아요 저희집이랑 너무 잘 어울리네요^^ 고급스러워보여요.


In [12]:
update

['유광 무광 정말 많이 고민했는데 유광선택할 잘 한거 같아요 저희집이랑 너무 잘 어울리네요^^ 고급스러워보여요.']

In [13]:
conn = pymysql.connect(host='localhost', user='root', password='root', db='collection', charset='utf8mb4')
cur = conn.cursor()
for data in update:
    cur.execute("insert into 29cm_lumir (product_no,review) values (650636,%s)", data)
conn.commit()
conn.close()

### 디비 저장 문제점
현재 디비에 저장하면 최근에 추가한 글이 가장 마지막에 들어간다. 그러므로 처음 클롤링한 데이터는 list reverse를 이용하여 저장한 하도록 하자(순서를 맞추기 위해서).   